In [1]:
import pandas as pd
import altair as alt

# Load the data
file_path = (
    "/Users/jakebarnabe/Desktop/projects/barns_power_rankings/Rankings_Table.csv"
)
df = pd.read_csv(file_path)

# Set 'Week' as the index
df = df.set_index("Week")

# Calculate the change in rankings for each team
ranking_changes = df.diff().fillna(0).astype(int)

# Reverse the signs: a lower ranking number (better) should be positive
ranking_changes_reversed = -ranking_changes

# Format the ranking change to show + for rise and - for fall
ranking_changes_formatted = ranking_changes_reversed.applymap(
    lambda x: f"+{x}" if x > 0 else (f"{x}" if x < 0 else "0")
)

# Melt the original DataFrame and the ranking changes DataFrame
df_reset = df.reset_index().melt(
    id_vars=["Week"], var_name="Team", value_name="Ranking"
)
ranking_changes_reset = ranking_changes_formatted.reset_index().melt(
    id_vars=["Week"], var_name="Team", value_name="Ranking Change"
)

# Merge the melted DataFrames
df_merged = pd.merge(df_reset, ranking_changes_reset, on=["Week", "Team"])

# Get the latest week
latest_week = df_merged["Week"].max()

# Create the line chart using Altair
lines = (
    alt.Chart(df_merged)
    .mark_line(point=True)
    .encode(
        x=alt.X("Week:O", title="Week", axis=alt.Axis(labelAngle=0)),
        y=alt.Y(
            "Ranking:Q", scale=alt.Scale(domain=[1, 10], reverse=True), title="Ranking"
        ),
        color="Team:N",
        tooltip=["Team", "Week", "Ranking", "Ranking Change"],
    )
    .properties(width=800, height=400)
)

# Add text labels to the most recent week's data points
text = (
    alt.Chart(df_merged[df_merged["Week"] == latest_week])
    .mark_text(align="left", dx=5, dy=-5)
    .encode(
        x=alt.X("Week:O"),
        y=alt.Y("Ranking:Q"),
        text="Team:N",
        color="Team:N",  # Keep team colors consistent
    )
)

# Combine the line chart with the text labels
chart = (
    (lines + text)
    .configure_legend(disable=True)  # Disable the legend at the top level
    .properties(title="Power Rankings Tracker")
)

# Save the chart
chart.save("../img/rankings_tracker.html")

/var/folders/8g/91jpwpd137ld1bq8pf74y58w0000gn/T/ipykernel_89112/1006077209.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ranking_changes_formatted = ranking_changes_reversed.applymap(


In [2]:
chart

alt.LayerChart(...)